In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [2]:
# train Parameters
# seq_length = 4
seq_length = 24
data_dim = 2
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 200

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화

# xy = np.loadtxt('./v3data/train_v3_data_cct.csv', delimiter=',')
# cct, cas_swr, 446to477, uvb, ptmt (892행 5열)
xy = np.loadtxt('./v3data/data2.csv', delimiter=',')

x = scaler.fit_transform(xy[:, [0, 1]]) # x = 맨 마지막 ptmt 제외 모든 것
y = scaler2.fit_transform(xy[:, [2]]) 

# build a dataset
dataX = []
dataY = []

# print(len(x))
# print(len(y))
# print(x[0])
# print(y[0])

for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)
#     print(np.shape(_x))

# print(train_size)
# print(test_size)
train_size = int(len(dataY) - 24)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

print(np.shape(trainX))

# print(np.shape(trainX))

(312, 24, 2)


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)


[step: 0] loss: 185.86099243164062
[step: 1] loss: 135.4376220703125
[step: 2] loss: 99.07176971435547
[step: 3] loss: 76.78659057617188
[step: 4] loss: 70.45518493652344
[step: 5] loss: 76.30339813232422
[step: 6] loss: 81.62855529785156
[step: 7] loss: 81.40160369873047
[step: 8] loss: 77.65081024169922
[step: 9] loss: 73.04732513427734
[step: 10] loss: 69.2232666015625
[step: 11] loss: 66.79263305664062
[step: 12] loss: 65.7138442993164
[step: 13] loss: 65.62071228027344
[step: 14] loss: 66.04625701904297
[step: 15] loss: 66.55865478515625
[step: 16] loss: 66.83866882324219
[step: 17] loss: 66.71289825439453
[step: 18] loss: 66.14936828613281
[step: 19] loss: 65.22634887695312
[step: 20] loss: 64.08866882324219
[step: 21] loss: 62.90377426147461
[step: 22] loss: 61.82448959350586
[step: 23] loss: 60.9610595703125
[step: 24] loss: 60.363616943359375
[step: 25] loss: 60.016109466552734
[step: 26] loss: 59.843421936035156
[step: 27] loss: 59.73265838623047
[step: 28] loss: 59.565532684

In [5]:
# Plot predictions
# plt.title(data + ' predict')
# plt.plot(dataY2, label='raw data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
# plt.ylabel("photometric")
plt.ylabel("cct")
# plt.ylabel("CAS_SWR")
# plt.ylabel("446to477")
# plt.ylabel("UV-B")
# plt.xticks(np.arange(0, 892, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
# plt.yticks(np.arange(0, 140000, step=10000)) # ptmt
# plt.yticks(np.arange(0, 60000, step=10000)) # cct
# plt.yticks(np.arange(0, 2.0, step=0.2)) # uvb
# plt.ylim(0, 100) #swr, 446to477
plt.grid(True)
# plt.legend(loc='upper right')
plt.show()